In [1]:
#Read the EMP FBMN mgf file
from pyteomics.mgf import MGF
mgf = MGF("bigdata/1907_EMPv2_INN_GNPS_derep_format.mgf")


In [2]:
#Get the QAC spectra
import pandas as pd
mgf.reset()
fmeta = pd.read_csv("fmeta_qac_id.csv")
qac_ms2 = [x for x in mgf if x['params']['scans'] in list(map(str, fmeta["X.featureID"].values.tolist()))]
len(qac_ms2)

223

In [3]:
def match_ppm(needle, haystack):
    if abs((needle - haystack)/needle*10**6) < 10:
        return(True)
    else:
        return False

In [5]:
match_ppm(60.08, 60.0801)

True

In [99]:
#Write QAC spectra in long table format
out = open('qac_ms2.txt',"w")
out.write("Scan Mz Intensity\n")
for s in qac_ms2:
    si = s['intensity array']/s['intensity array'].sum() 
    si = si[si > 1/100] 
    for m, i in zip(s['m/z array'], si):
        out.write("%s %s %s\n" % (s['params']['scans'], m,i) )
out.close()

In [15]:
#Write likely QAC (based on the 60.08 fragment) in long format
mgf.reset()
out = open('qac_ms2_fragment.txt',"w")
out.write("Scan Mz Intensity\n")
fragments = []
for s in mgf:
    si = s['intensity array']/s['intensity array'].sum() 
    si = si[si > 1/10] 
    for m in s['m/z array']:
        if  match_ppm(60.0808, m):
            fragments.append(s)
            for m, i in zip(s['m/z array'], si):
                out.write("%s %s %s\n" % (s['params']['scans'], m,i) )
            continue
    
out.close()

In [16]:
len(fragments) # or

2844

In [137]:
qacf_scans = [x['params']['scans'] for x in fragments]
qac_scans = [x['params']['scans'] for x in qac_ms2]
print(len(qac_scans), len(qacf_scans))

223 3097


In [140]:
from networkx import read_graphml, write_graphml
G = read_graphml("bigdata/FEATURE-BASED-MOLECULAR-NETWORKING-3381f4f4-download_cytoscape_data-main.graphml")

In [148]:
flatten = lambda t: [item for sublist in t for item in sublist]
Gqac = G.subgraph(qac_scans)
Gqacf = G.subgraph(qacf_scans)
Gqac_neighbours = G.subgraph(flatten([G.neighbors(x) for x in qac_scans]))
for n in Gqac_neighbours:
    if n in qac_scans:
        G.nodes[n]["QAC_label"] = "QAC"
    else:
        G.nodes[n]["QAC_label"] = "Neighbour"

write_graphml(Gqac, "qac.graphml")
write_graphml(Gqac_neighbours, "Gqac_neighbours.graphml")

In [145]:
[x for x in G.neighbors(qac_scans[1])]

['40274']

In [157]:
mgf.reset()
out = open('tpl_fragments.txt',"w")
out.write("Scan Mz Intensity\n")
fragments = []
for s in mgf:
    si = s['intensity array']/s['intensity array'].sum() 
    si = si[si > 1/10] 
    for m in s['m/z array']:
        if m > 119.99 and m < 197.01:
            fragments.append(s)
            for m, i in zip(s['m/z array'], si):
                out.write("%s %s %s %s\n" % (s['params']['scans'], m,i, s['params']['pepmass'][0] ) )
            continue
    
out.close()

In [154]:
len(fragments)

68

In [161]:
mgf.reset()
[x for x in mgf if x['params']['pepmass'][0] < 386.1 and x['params']['pepmass'][0] > 385.9] 

[{'params': {'feature_id': '42755',
   'pepmass': (385.9245, None),
   'corr_groupid': '306',
   'scans': '42755',
   'rtinseconds': 157.363 second,
   'charge': [1],
   'mslevel': '2'},
  'm/z array': array([ 51.613 ,  52.4764,  53.9042,  58.6963,  60.056 ,  61.2561,
          76.1206,  78.6495,  80.472 ,  86.035 ,  96.0554, 120.0553,
         137.0817, 182.3574, 185.0581, 185.0817, 186.0659, 188.069 ,
         200.0663, 200.0931, 201.0767, 211.0605, 226.0723, 227.0798,
         228.0878, 246.0978, 250.9679, 254.9773, 262.9919, 263.9764,
         264.0003, 264.9831, 277.9793, 278.9869, 283.1467, 289.9792,
         305.9982, 307.0059, 326.8759, 327.8822, 342.2198, 342.9169,
         343.903 , 356.8976, 356.9424, 368.8989, 374.908 , 385.9241,
         386.1796, 386.908 , 391.6662]),
  'intensity array': array([  2300.,   2000.,   2400.,   2200.,  17000.,   2000.,   2400.,
           2000.,   2400.,  17000.,   6500.,  10000.,  15000.,   2500.,
          11000.,  11000.,  68000.,   6100.,